# **CHAPTER 1. Pangenome analysis**

Import all the modules needed

In [1]:
import os
import re
import csv
import pandas as pd
from Bio import Entrez, SeqIO

In this study `PanACoTA` software will be used for Pangenome analysis

First, create a directory to store all the data

In [2]:
%%bash

mkdir pangenome/
mkdir pangenome/data/
mkdir pangenome/Annotation/
mkdir pangenome/Annotation/Genes/
mkdir pangenome/Annotation/Proteins/
mkdir pangenome/Annotation/Proteins_classic/

Then we will need the list of accession numbers of _Streptomyces albidoflavus_ complete genomes

In [3]:
! esearch -db nucleotide \
    -query '("Streptomyces albidoflavus"[Organism] OR "Streptomyces albidoflavus"[All Fields]) AND "complete"[All Fields] AND (bacteria[filter] AND biomol_genomic[PROP] AND refseq[filter] AND is_nuccore[filter] AND ("5000000"[SLEN] : "20000000"[SLEN]))' \
    | efetch -format acc > pangenome/data/accession_numbers.txt

This function will download us everything

In [4]:
def get_sequences(email, file_path, output_dir, format, extension):
    Entrez.email = email

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Read accession numbers from file
    with open(file_path, "r") as file:
        accession_numbers = file.read().split()

    def download_sequence(accession):
        """Fetches a single sequence from NCBI and saves it as a FASTA file."""
        try:
            handle = Entrez.efetch(
                db="nucleotide", id=accession, rettype=format, retmode="text"
            )
            records = list(SeqIO.parse(handle, "fasta"))  # Use parse() instead of read()
            handle.close()

            if records:
                output_path = os.path.join(output_dir, f"{accession.split('.')[0]}.{extension}")
                SeqIO.write(records, output_path, "fasta")
                print(f"Downloaded: {accession}")
            else:
                print(f"No CDS found for {accession}")

        except Exception as e:
            print(f"Failed to download {accession}: {e}")

    # Download sequences for each accession number
    for accession in accession_numbers:
        download_sequence(accession)

    print("All downloads completed.")

In [5]:
email = "ivpopov@donstu.ru"
accession_numbers = "pangenome/data/accession_numbers.txt"

Download genomes

In [ ]:
get_sequences(email,
              accession_numbers,
              "pangenome/Annotation/Genes",
              format = "fasta_cds_na",
              extension = "gen")

Download proteomes

In [ ]:
get_sequences(email,
              accession_numbers,
              "pangenome/Annotation/Proteins_classic",
              format = "fasta_cds_aa",
              extension = "prt")

Now, we need to rename downloaded proteomes to make them face the requirements of `PanACoTA` input

In [8]:
%%bash

for file in pangenome/Annotation/Proteins_classic/*.prt; do 
    awk '
    {
        if ($0 ~ /^>/) {
            line = $0
            sub(/^>lcl\|/, ">", line)                       # Step 1: Remove "lcl|"
            split(line, a, "_prot_")                        # Step 2: Split into before/after "_prot_"
            id = a[1]                                       # Take the part before _prot_
            sub(/\.[0-9]+$/, "", id)                        # Step 3: Remove .1/.11/.111 at end of ID

            # Now process the part after _prot_ (if exists)
            if (length(a) > 1) {
                split(a[2], b, "_")                         # Split second part by "_"
                seq = b[2]                                  # Take second chunk: "RT032785929" or similar
                gsub(/[^0-9]/, "", seq)                     # Step 5: Remove letters, keep numbers only
                print id "_" seq
            } else {
                print id
            }
        } else {
            print
        }
    }' "$file" > pangenome/Annotation/Proteins/$(basename "$file")
done

Now we apply the renaming from proteomes to genomes!

In [ ]:
# Define directories
proteins_dir = "pangenome/Annotation/Proteins"
genes_dir = "pangenome/Annotation/Genes"

# Ensure Genes directory exists
if not os.path.exists(genes_dir):
    print("Genes directory does not exist.")
    exit(1)

# Function to extract FASTA sequences
def read_fasta(file_path):
    sequences = []
    with open(file_path, "r") as f:
        seq = []
        header = None
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if header:
                    sequences.append((header, "\n".join(seq)))
                header = line  # Store new header
                seq = []
            else:
                seq.append(line)
        if header:
            sequences.append((header, "\n".join(seq)))  # Append last sequence
    return sequences

# Function to write updated FASTA sequences
def write_fasta(file_path, sequences):
    with open(file_path, "w") as f:
        for header, seq in sequences:
            f.write(f"{header}\n{seq}\n")

# Process all .prt files in Proteins directory
for prt_file in os.listdir(proteins_dir):
    if prt_file.endswith(".prt"):
        # Get corresponding .gen file
        base_name = os.path.splitext(prt_file)[0]  # Remove .prt extension
        gen_file = f"{base_name}.gen"
        
        prt_path = os.path.join(proteins_dir, prt_file)
        gen_path = os.path.join(genes_dir, gen_file)

        # Check if corresponding .gen file exists
        if not os.path.exists(gen_path):
            print(f"Skipping {gen_file} (not found in Genes directory)")
            continue

        # Read sequences from .prt and .gen files
        prt_seqs = read_fasta(prt_path)
        gen_seqs = read_fasta(gen_path)

        # Ensure both files have the same number of sequences
        if len(prt_seqs) != len(gen_seqs):
            print(f"Skipping {gen_file} (mismatch: {len(prt_seqs)} protein seqs vs {len(gen_seqs)} gene seqs)")
            continue

        # Replace headers in .gen file
        updated_gen_seqs = [(prt_seqs[i][0], gen_seqs[i][1]) for i in range(len(gen_seqs))]

        # Write updated .gen file
        write_fasta(gen_path, updated_gen_seqs)
        print(f"Updated {gen_file} with new headers from {prt_file}")

print("Processing complete.")

Now create a list file with the proteomes to build the pangenome

In [10]:
! ls pangenome/Annotation/Proteins/*.prt | sed 's|pangenome/Annotation/Proteins/||' | sed 's/\.prt$//' >\
    pangenome/Annotation/LSTINFO-.lst

Also, we must create a merged proteomes dataset

In [11]:
! cat pangenome/Annotation/Proteins/* > pangenome/Annotation/Proteins/StAl.All.prt

Good! Now let's construct a pangenome with the proteins identity setting = `0.9` (90%).

In [12]:
! PanACoTA pangenome -l pangenome/Annotation/LSTINFO-.lst -n StAl -d pangenome/Annotation/Proteins/ -o pangenome/Pangenome -i 0.9

  * [2025-04-29 15:50:39] : INFO  PanACoTA version 1.4.0
  * [2025-04-29 15:50:39] : INFO  Command used
 	 > PanACoTA pangenome -l pangenome/Annotation/LSTINFO-.lst -n StAl -d pangenome/Annotation/Proteins/ -o pangenome/Pangenome -i 0.9
  * [2025-04-29 15:50:39] : INFO  Will run MMseqs2 with:
	- minimum sequence identity = 90.0%
	- cluster mode 1
  * [2025-04-29 15:50:39] : INFO  Creating database
|       ◐              |  -  Elapsed Time: 0:00:00
  * [2025-04-29 15:50:40] : INFO  Clustering proteins...
|        ◐             |  -  Elapsed Time: 0:04:18
  * [2025-04-29 15:54:59] : INFO  Converting mmseqs results to pangenome file
  * [2025-04-29 15:54:59] : INFO  Pangenome has 12027 families.
  * [2025-04-29 15:54:59] : INFO  Retrieving information from pan families
  * [2025-04-29 15:54:59] : INFO  Generating qualitative and quantitative matrix, and summary file
  * [2025-04-29 15:54:59] : INFO  DONE


The function below will calculate the pangenome composition for:<br>
- _Streptomyces albidoflavus_<br>
- _Streptomyces albidoflavus_ SM254

In [ ]:
def analyze_pangenome(input_path: str, output_path: str, target_prefix: str = None, total_genomes: int = 34) -> None:
    """Analyze pangenome gene clusters and categorize them by presence across genomes."""
    counts = {
        'Core': 0,
        'Soft-core': 0,
        'Shell': 0,
        'Cloud': 0
    }

    with open(input_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if not parts:
                continue

            elements = parts[1:]

            # If filtering by prefix, check and ensure no duplicates
            if target_prefix:
                if not any(p.startswith(target_prefix) for p in elements):
                    continue
                if len(set(elements)) != len(elements):
                    continue
            else:
                if len(set(elements)) != len(elements):
                    continue

            n = len(elements)

            if n == total_genomes:
                counts['Core'] += 1
            elif n in {total_genomes - 1, total_genomes - 2}:
                counts['Soft-core'] += 1
            elif 1 < n < total_genomes - 2:
                counts['Shell'] += 1
            elif n <= 1:
                counts['Cloud'] += 1

    with open(output_path, 'w', newline='') as out_file:
        writer = csv.writer(out_file, delimiter='\t')
        writer.writerow(['Category', 'Count'])
        for category, count in counts.items():
            writer.writerow([category, count])

    print("Analysis complete. Results saved to:", output_path)

In [4]:
# Run for SM254 (with filtering for NZ_CP014485_)
analyze_pangenome(
    input_path='pangenome/Pangenome/PanGenome-StAl.All.prt-clust-0.9-mode1.lst',
    output_path='pangenome/sm254_pangenome.tsv',
    target_prefix='NZ_CP014485_'
)

# Run for all StAl (no filtering)
analyze_pangenome(
    input_path='pangenome/Pangenome/PanGenome-StAl.All.prt-clust-0.9-mode1.lst',
    output_path='pangenome/StAl_pangenome.tsv',
    target_prefix=None
)

Analysis complete. Results saved to: pangenome/sm254_pangenome.tsv
Analysis complete. Results saved to: pangenome/StAl_pangenome.tsv


For now please proceed to the `02_pangenome_visualization.R` and run the analysis there. Then come back!

Perfect! Now run `PanACoTA`'s `corepers` module to extract core genes!

In [13]:
! PanACoTA corepers -p pangenome/Pangenome/PanGenome-StAl.All.prt-clust-0.9-mode1.lst -o pangenome/Coregenome -t 1

  * [2025-04-29 15:55:43] : INFO  PanACoTA version 1.4.0
  * [2025-04-29 15:55:43] : INFO  Command used
 	 > PanACoTA corepers -p pangenome/Pangenome/PanGenome-StAl.All.prt-clust-0.9-mode1.lst -o pangenome/Coregenome -t 1
  * [2025-04-29 15:55:43] : INFO  Will generate a CoreGenome.
  * [2025-04-29 15:55:43] : INFO  Retrieving info from binary file
  * [2025-04-29 15:55:43] : INFO  Generating Persistent genome of a dataset containing 34 genomes
  * [2025-04-29 15:55:43] : INFO  The core genome contains 3867 families, each one having exactly 34 members, from the 34 different genomes.
  * [2025-04-29 15:55:43] : INFO  Persistent genome step done.


Now we have to perform multiple sequences alignment of 3867 core genes<br>
>Lifehack: instead of running `MAFFT` by ourselves, we can still run `PanACoTA`!

In [16]:
! PanACoTA align -c pangenome/Coregenome/PersGenome_PanGenome-StAl.All.prt-clust-0.9-mode1.lst-all_1.0.lst\
    -l pangenome/Annotation/LSTINFO-.lst -n StAl -d pangenome/Annotation/ -o pangenome/Alignment

  * [2025-04-29 16:03:15] : INFO  PanACoTA version 1.4.0
  * [2025-04-29 16:03:15] : INFO  Command used
 	 > PanACoTA align -c pangenome/Coregenome/PersGenome_PanGenome-StAl.All.prt-clust-0.9-mode1.lst-all_1.0.lst -l pangenome/Annotation/LSTINFO-.lst -n StAl -d pangenome/Annotation/ -o pangenome/Alignment
  * [2025-04-29 16:03:15] : INFO  Found 34 genomes.
  * [2025-04-29 16:03:15] : INFO  Reading PersGenome and constructing lists of missing genomes in each family.
  * [2025-04-29 16:03:15] : INFO  Getting all persistent proteins and classify by strain.
  * [2025-04-29 16:03:16] : INFO  Extracting proteins and genes from all genomes
Extraction:████████████████ 34/34 (100%) - Elapsed Time: 0:00:18 Time:  0:00:18
  * [2025-04-29 16:03:34] : INFO  Starting alignment of all families: protein alignment, back-translation to nucleotides, and add missing genomes in the family
Alignment: ████████████████████████ 3867/3867 (100%) - Elapsed Time: 1:17:28 -   * [2025-04-29 17:21:03] : INFO  Concat

We've got the MSAs! Now what? `MODELFINDER`!

First, create a directory to store `ModelFinder` log files

In [17]:
! mkdir pangenome/model-finder/

Then run a `ModelFinder` on concatenated MSA!

In [ ]:
%%bash

iqtree2 -m MFP -s pangenome/Alignment/Phylo-StAl/StAl.nucl.grp.aln --prefix pangenome/model-finder/StAl -T AUTO -safe

Get the best fit model

In [1]:
! head -42 pangenome/model-finder/StAl.iqtree | tail -6

Best-fit model according to BIC: TVM+F+R5

List of models sorted by BIC scores: 

Model                  LogL         AIC      w-AIC        AICc     w-AICc         BIC      w-BIC
TVM+F+R5        -7526742.522 15053645.044 +    0.554 15053645.047 +    0.554 15054697.419 +    0.999


At last... We have the best fit model... It's time to launch `IQ-TREE`!

First, create a directory to store the tree

In [2]:
! mkdir pangenome/tree/

Then run an `IQ-TREE` on concatenated MSA

In [ ]:
%%bash

iqtree2 -s pangenome/Alignment/Phylo-StAl/StAl.nucl.grp.aln -m TVM+F+R5 -pre pangenome/tree/StAl_ufb -bb 10000 -nt AUTO

Actually that's all!<br>
But now we'll fetch metadata on _Streptomyces albidoflavus_ from NCBI RefSeq<br>
It will be used to annotate the trees in `ggtree`!

First create a directory where to store metadata

In [4]:
! mkdir metadata/

Now fetch metadata!

In [1]:
! Phyloki --fetch_metadata -email ivpopov@donstu.ru -i pangenome/data/accession_numbers.txt -o metadata/metadata.tsv

Metadata retrieval complete.
File saved to metadata/metadata.tsv


And the last thing to do — trim versions out of accession numbers (to make metadata meet the corresponding samples in the tree!)

In [ ]:
df = pd.read_csv('metadata/metadata.tsv', sep='\t')
df["AN"] = df["AN"].str.split('.').str[0]
df.to_csv('metadata/welldone_metadata.tsv', sep='\t', index=False)

That's all for pangenome analysis! Please proceed to the `03_ggtree_journal` for further analysis!

And then please welcome to `04_ANI.ipynb` for Average Nucleotide Identity analysis!